# Final assignment 
## Language switching experiment

### Things to do:
- Create a _PsychoPy experiment_ to collect _response time_ data
- Store the RTs and accuracy in a _csv file_
- _Plot_ the data using seaborn
- _Analyze_ the data using PyMC3 or Statsmodels
- __Commit__ all of your changes regularly 

## 1. Creating the experiment
### 1.1. Stimuli

The stimuli will consist of pictures, presented on one side of the screen. When a pictures is presented on the _left_ side of the screen, the picture needs to be named in L1. When a picture is presented on the _right_ side of the screen, the picture needs to be named in the L2. 

So, for the stimuli I need pictures stored in a separate subfolder. 

In [1]:
from PIL import Image

Now, create a Psychopy experiment. I created a file called `switch.py`. First of all, to open a window:

In [10]:
'''
from psychopy import core, visual, event 
win = visual.Window(color = 'white', fullscr = True) #Open a window with a white background, full screen instead of pixels 
core.wait(5)  
win.close()  
'''

"\nfrom psychopy import core, visual, event \nwin = visual.Window(color = 'white', fullscr = True) #Open a window with a white background, full screen instead of pixels \ncore.wait(5)  # wait for 5 seconds so we can look at the window\nwin.close()  # close the window\n"

I would like to include an introduction screen with written instructions for the participants. 

In [ ]:
import subprocess as sp

sp.run(['python', 'switch.py'])

### 1.2 Responses 

The participants need to _name_ the pictures, meaning that the responses will consist of a voice key. Figure out how to do this __here__. However, the first step is probably importing _sound device_, as this can be used to record voices. 

In [ ]:
import sounddevice as sd